Use this utlity to update the returns and std_dev fields within investment-options.csv

In [22]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [24]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [25]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [26]:
# Read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# Remove the exchange from the beginning of the symbol.
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()

dow30['Symbol'] = dow30.apply(_symbol, axis=1)
dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AMZN,Amazon,Retailing
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology


In [27]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Biopharmaceutical': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Drink industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services',
 'Home Improvement': 'US Stocks:Consumer Discretionary',
 'Semiconductor industry': 'US Stocks:Technology',
 'Clothing industry': 'US Sto

In [28]:
# Map dow30 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
dow30.index = dow30.index.str.replace('.', '-', regex=False)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AMZN,Amazon,US Stocks:Consumer Discretionary
AAPL,Apple,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar,US Stocks:Industrials
CVX,Chevron,US Stocks:Energy
CSCO,Cisco,US Stocks:Technology


In [29]:
# Make symbols list.
symbols = list(dow30.index)
#symbols

In [30]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MMM AXP AMGN AMZN AAPL BA CAT CVX CSCO KO DIS GS HD HON IBM JNJ JPM MCD MRK MSFT NKE NVDA PG CRM SHW TRV UNH VZ V WMT 


In [31]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AMZN,AAPL,BA,CAT,CVX,CSCO,KO,...,NKE,NVDA,PG,CRM,SHW,TRV,UNH,VZ,V,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,98.44,80.13,121.19,15.43,24.35,113.66,70.37,74.21,20.19,30.89,...,42.35,0.48,68.28,58.91,79.78,84.12,86.56,28.40,61.89,23.44
2015-01-05,96.22,78.01,119.75,15.11,23.66,112.87,66.65,71.25,19.79,30.89,...,41.67,0.47,67.95,57.85,79.54,83.11,85.14,28.17,60.53,23.37
2015-01-06,95.20,76.35,115.89,14.76,23.66,111.54,66.22,71.21,19.78,31.13,...,41.43,0.46,67.64,56.88,79.30,82.37,84.96,28.45,60.14,23.55
2015-01-07,95.89,78.02,119.94,14.92,24.00,113.27,67.25,71.15,19.96,31.52,...,42.28,0.46,68.00,56.61,80.65,83.77,85.83,28.27,60.94,24.18
2015-01-08,98.19,79.13,119.51,15.02,24.92,115.28,67.94,72.78,20.12,31.90,...,43.26,0.48,68.77,58.26,82.86,85.51,89.93,28.87,61.76,24.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,130.18,300.34,262.65,223.75,255.59,180.72,364.86,144.00,59.21,62.45,...,76.42,137.01,169.53,338.45,343.29,241.41,509.99,39.92,318.66,91.66
2024-12-30,129.13,296.63,259.30,221.30,252.20,176.55,363.01,143.07,58.79,62.03,...,74.65,137.49,167.09,335.74,338.75,239.87,507.80,39.60,315.31,90.57
2024-12-31,129.09,296.09,260.64,219.39,250.42,177.00,362.76,144.84,58.80,62.26,...,75.67,134.29,167.65,334.33,339.93,240.89,505.86,39.99,316.04,90.35


In [32]:
# Sample symbol.
symbol = 'MMM'

In [33]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(48.30525530206151)

In [34]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [35]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

np.float64(0.16209549224489256)

In [36]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.09079785544163718)

In [37]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31    97.39
2015-02-28   101.83
2015-03-31    99.59
2015-04-30    94.43
2015-05-31    96.66
2015-06-30    93.75
2015-07-31    91.95
2015-08-31    86.97
2015-09-30    86.74
2015-10-31    96.19
2015-11-30    96.43
2015-12-31    92.77
2016-01-31    92.99
2016-02-29    97.31
2016-03-31   103.36
2016-04-30   103.83
2016-05-31   105.10
2016-06-30   109.35
2016-07-31   111.37
2016-08-31   112.62
2016-09-30   110.73
2016-10-31   103.86
2016-11-30   108.60
2016-12-31   112.92
2017-01-31   110.55
2017-02-28   118.61
2017-03-31   121.77
2017-04-30   124.64
2017-05-31   130.91
2017-06-30   133.30
2017-07-31   128.80
2017-08-31   131.57
2017-09-30   135.16
2017-10-31   148.23
2017-11-30   157.36
2017-12-31   152.33
2018-01-31   162.12
2018-02-28   153.32
2018-03-31   142.91
2018-04-30   126.55
2018-05-31   129.28
2018-06-30   128.94
2018-07-31   139.17
2018-08-31   139.18
2018-09-30   139.04
2018-10-31   125.54
2018-11-30   138.13
2018-12-31   126.59
2019-01-31   133.07
2019-02-28   13

In [38]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2024-09-30    0.01
2024-10-31   -0.06
2024-11-30    0.04
2024-12-31   -0.03
2025-01-31    0.01
Freq: ME, Name: MMM, Length: 121, dtype: float64

In [39]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

np.float64(0.3092575211557992)

In [40]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [41]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [42]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')